In [1]:

import pandas as pd
import numpy as np
import json
import requests
from math import radians, sqrt, sin, cos, atan2

dirpath = 'C:/Users/thuang1/Desktop/Snowplow/'


test = pd.read_csv(dirpath+"15min_test.csv", header=0)

In [2]:
test.head(2)

,LABEL,XPOSITION,YPOSITION,HEADING,VELOCITY,ROADTEMP,AIRTEMP,SOLIDMATERIAL,LIQUIDMATERIAL,PREWETMATERIAL,...,UNDERBELLYPLOWSTATE,ODOMETER,SOLIDRUNTOTAL,LIQUIDRUNTOTAL,PREWETRUNTOTAL,SOLIDSETRATE,LIQUIDSETRATE,PREWETSETRATE,BLASTMODE,LOGDT_LOCAL
0,A33831,-92.101799,41.692169,276,96.304,NaN,NaN,0,0,0,...,0,NaN,50411,12693,0,0,0,0.0,NaN,20161211160000
1,A32592,-90.524384,41.807499,86,22.224,NaN,NaN,0,0,0,...,0,NaN,161976,67229,0,0,0,0.0,NaN,20161211160000


In [3]:
def geocalc(lat1, lon1, lat2, lon2):
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon1 - lon2

    EARTH_R = 6372.8*3280.84 # km to feet

    y = sqrt(
        (cos(lat2) * sin(dlon)) ** 2
        + (cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)) ** 2
        )
    x = sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(dlon)
    c = atan2(y, x)
    return EARTH_R * c

def get_locations(df):
    d = []
    for i in xrange(len(df)):
#        x = {'geometry':{'y':df.iloc[i]['Latitude'], 'x':df.iloc[i]['Longitude']}}
        x = {'geometry':{'y':df.iloc[i]['YPOSITION'], 'x':df.iloc[i]['XPOSITION']}}
        d.append(x)

    locations = json.dumps(d, ensure_ascii=False)
    return d, locations


def get_routeId(d, dat):
    
            
    xlong = d['geometry']['x']
    xlati = d['geometry']['y']
    
    df2 = pd.DataFrame(dat, columns=['routeId','long','lati','measure'])
    
    df2['route_class']=df2.routeId.str[0]

    df2['ramp'] = 1
    df2.loc[(pd.isnull(df2.routeId.str[11])),'ramp'] = 0
    
    df2['dist']=df2.apply(lambda x: geocalc(float(x['lati']),float(x['long']),xlati,xlong), axis=1)
    
    df2['priority'] = 1 # low
    
    df2.loc[(df2.dist<=20) & (df2.ramp==0), 'priority'] = 0 # high
  
  
    df2 = df2.sort_values(by=['dist']).reset_index(drop=True)
    
    if df2.loc[0,'ramp']==1:
        df2 = df2.sort_values(by=['priority','dist']).reset_index(drop=True)
 
    routeId = df2.loc[0,'routeId']
    measure = df2.loc[0,'measure']

    return np.array([routeId, measure])



def get_measure(locations, tolerance):
       

    url = 'https://gis.iowadot.gov/ramsa/rest/services/lrs/MapServer/exts/LRSServer/networkLayers/0/geometryToMeasure?'
    data = {'f':'json','locations': locations,'tolerance':str(tolerance),'inSR':'4326'}
    response = requests.post(url, data=data)
    r=response.json()

        
        
    # extract results
    resall=[]
    for i in r['locations']:
        res=[]
        if i['status']==u'esriLocatingCannotFindLocation':
            routeid="0"
            x=0
            y=0
            measure=0
            res.append([routeid, x, y, measure])
        else:

            for j in i['results']:
                routeid=j['routeId'].encode('ascii','ignore')
                x=j['geometry']['x']
                y=j['geometry']['y']
                measure=j['measure']
                res.append([routeid, x, y, measure])
        resall.append(res)
    
    datall=[]
    dat=[]

    for i in xrange(len(resall)):

        if resall[i][0][0]=='0':
            dat = [None, None]
        else:
            dat = get_routeId(d[i], resall[i])
        datall.append(dat)

    datall=np.array(datall)


    
    
    return datall

In [4]:
n=1000

ans=[]
for i in xrange(len(test)/n+1):
     ans.append(test[i*n:i*n+n])

In [5]:
# Main

from datetime import datetime
now1=datetime.now()

aa=[]
i=0

for a in ans:
    i=i+1
    d, locations = get_locations(a)
    
    datall = get_measure(locations, 50)    

    a['routeId']=datall[:,0]
    a['measure']=datall[:,1]
    
    aa.append(a)
    
    print('Success! Progress: '+str(round(float(i)/len(ans)*100,2))+"%")

aa=pd.concat(aa,axis=0).reset_index(drop=True)

now2=datetime.now()

print('Runtime: '+str(now2-now1))

C:\Users\thuang1\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\thuang1\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Success! Progress: 7.14%
Success! Progress: 14.29%
Success! Progress: 21.43%
Success! Progress: 28.57%
Success! Progress: 35.71%
Success! Progress: 42.86%
Success! Progress: 50.0%
Success! Progress: 57.14%
Success! Progress: 64.29%
Success! Progress: 71.43%
Success! Progress: 78.57%
Success! Progress: 85.71%
Success! Progress: 92.86%
Success! Progress: 100.0%
Runtime: 0:15:36.817000


In [6]:
aa.head(4)

,LABEL,XPOSITION,YPOSITION,HEADING,VELOCITY,ROADTEMP,AIRTEMP,SOLIDMATERIAL,LIQUIDMATERIAL,PREWETMATERIAL,...,SOLIDRUNTOTAL,LIQUIDRUNTOTAL,PREWETRUNTOTAL,SOLIDSETRATE,LIQUIDSETRATE,PREWETSETRATE,BLASTMODE,LOGDT_LOCAL,routeId,measure
0,A33831,-92.101799,41.692169,276,96.304,NaN,NaN,0,0,0,...,50411,12693,0,0,0,0.0,NaN,20161211160000,S001910080W,91.7564130376
1,A32592,-90.524384,41.807499,86,22.224,NaN,NaN,0,0,0,...,161976,67229,0,0,0,0.0,NaN,20161211160000,S001920030E,312.061062045
2,A32542,-91.110817,42.502361,44,1.852,NaN,NaN,0,0,0,...,78641,6001,0,0,0,0.0,NaN,20161211160000,None,None
3,A32890,-91.055656,42.459461,292,61.116,NaN,NaN,0,0,0,...,67435,15824,0,0,0,0.0,NaN,20161211160000,S001920020W,23.02497912


In [7]:
len(aa)

13333

In [8]:
len(test)

13333

In [9]:
aa = aa.sort_values(by=['LABEL','LOGDT_LOCAL']).reset_index(drop=True)

In [10]:
aa_backup_1 = aa.copy()

In [11]:
aa.to_csv('15min_raw_request_res.csv', index=False)

In [21]:
aa = aa_backup_1.copy()

## route correction

In [22]:
def route_correction(a):
        
    a['route_class']=a.routeId.str[0:10]
    a['END_ROUTECLASS']=a.END_ROUTEID.str[0:10] 
    a.loc[~pd.isnull(a.routeId.str[11]),'route_class']='R'
    a.loc[~pd.isnull(a.END_ROUTEID.str[11]),'END_ROUTECLASS']='R'
    a['diff']=0
    a.loc[a['END_ROUTECLASS']!=a['route_class'],'diff']=1
    a['NEXT_END']=a['END_ROUTECLASS'].shift(-1)
    a['NEXT_LABEL']=a['LABEL'].shift(-1)
    index = a.index[(a['diff']==1) & (a['route_class']==a['NEXT_END']) & (a['NEXT_LABEL']==a['LABEL'])]
    
    a.loc[index,'END_ROUTEID']=a.loc[index, 'routeId']
    a.loc[index,'END_ROUTECLASS']=a.loc[index, 'route_class']
    
    a.loc[index+1,'routeId']=a.loc[index+1, 'END_ROUTEID']
    a.loc[index+1,'route_class']=a.loc[index+1, 'END_ROUTECLASS']
    
    a['route_correction']=0
    a.loc[index, 'route_correction']=1 
    a.loc[index+1, 'route_correction']=1
    
    a = a.loc[a['route_class']==a['END_ROUTECLASS']]
    
    a=a.drop(['END_ROUTECLASS','route_class','NEXT_END','NEXT_LABEL','diff'],axis=1)
    
    return a

In [23]:
def route_process(a):
    a = a[pd.notnull(a['routeId'])]
    a = a.sort_values(by=['LABEL','LOGDT_LOCAL']).reset_index(drop=True)
    a['END_LABEL']=a['LABEL'].shift(-1)
    a['END_LATI']=a['YPOSITION'].shift(-1)
    a['END_LONG']=a['XPOSITION'].shift(-1)
    a['END_DT']=a['LOGDT_LOCAL'].shift(-1).apply(lambda x: '%.0f'%x)
    a['END_ROUTEID']=a['routeId'].shift(-1)
    a['END_MEASURE']=a['measure'].shift(-1)
    a = a.loc[a['LABEL']==a['END_LABEL']]
    
    a = route_correction(a)
    
    return a

In [24]:
aa = route_process(aa)

In [25]:
len(aa)

10401

In [26]:
aa.to_csv('route_corrected.csv', index = False)

In [27]:
def get_measure_w_route_start(df):
    

    route='routeId'
    xx=df['XPOSITION']
    yy=df['YPOSITION']


    
    d = []
    loc = {'geometry':{'y':yy, 'x':xx}, 'routeId':df[route]}
    d.append(loc)

    locations = json.dumps(d, ensure_ascii=False)
       

    url = 'https://gis.iowadot.gov/ramsa/rest/services/lrs/MapServer/exts/LRSServer/networkLayers/0/geometryToMeasure?'
    tolerance = 50
    
    data = {'f':'json','locations': locations,'tolerance':str(tolerance),'inSR':'4326'}
    response = requests.post(url, data=data)
    r=response.json()
    
    if r['locations'][0]['status']!=u'esriLocatingOK':
        dat=None
    else:
        dat=(r['locations'][0]['results'][0]['measure'])

    
    return dat


In [28]:
def get_measure_w_route_end(df):
    

    route='END_ROUTEID'
    xx=df['END_LONG']
    yy=df['END_LATI']
            

    
    d = []
    loc = {'geometry':{'y':yy, 'x':xx}, 'routeId':df[route]}
    d.append(loc)

    locations = json.dumps(d, ensure_ascii=False)
       

    url = 'https://gis.iowadot.gov/ramsa/rest/services/lrs/MapServer/exts/LRSServer/networkLayers/0/geometryToMeasure?'
    tolerance = 50
    
    data = {'f':'json','locations': locations,'tolerance':str(tolerance),'inSR':'4326'}
    response = requests.post(url, data=data)
    r=response.json()
    
    if r['locations'][0]['status']!=u'esriLocatingOK':
        dat=None
    else:
        dat=(r['locations'][0]['results'][0]['measure'])

    
    return dat


In [29]:
def route_measure_correction(a):
    a.loc[a['route_correction']==1,'measure']=a.loc[a['route_correction']==1].apply(get_measure_w_route_start, axis=1)
    a.loc[a['route_correction']==1,'END_MEASURE']=a.loc[a['route_correction']==1].apply(get_measure_w_route_end, axis=1)
  
    return a

In [30]:
aa = route_measure_correction(aa)

In [31]:
aa.tail(5)

,LABEL,XPOSITION,YPOSITION,HEADING,VELOCITY,ROADTEMP,AIRTEMP,SOLIDMATERIAL,LIQUIDMATERIAL,PREWETMATERIAL,...,LOGDT_LOCAL,routeId,measure,END_LABEL,END_LATI,END_LONG,END_DT,END_ROUTEID,END_MEASURE,route_correction
10924,A34393,-93.490700,41.150139,313,64.820,NaN,NaN,0,0,0,...,20161211161510,S001920065N,41.2213749886,A34393,41.151470,-93.492607,20161211161522,S001920065N,41.3567736428,0
10925,A34393,-93.492607,41.151470,313,64.820,NaN,NaN,0,0,0,...,20161211161522,S001920065N,41.3567736428,A34393,41.152691,-93.494324,20161211161533,S001920065N,41.4796119518,0
10926,A34393,-93.494324,41.152691,313,62.968,NaN,NaN,0,0,0,...,20161211161533,S001920065N,41.4796119518,A34393,41.153912,-93.496040,20161211161544,S001920065N,41.6025475543,0
10927,A34393,-93.496040,41.153912,313,64.820,NaN,NaN,0,0,0,...,20161211161544,S001920065N,41.6025475543,A34393,41.155048,-93.497627,20161211161554,S001920065N,41.7165728338,0
10929,B44042T,-91.609207,41.710369,60,9.260,NaN,NaN,0,0,0,...,20161211160800,I072441010N,0.137587994185,B44042T,41.710659,-91.609131,20161211160811,I072441010N,0.157640632914,0


In [32]:
aa_backup_2 = aa.copy()

In [33]:
aa.to_csv('route_correction.csv', index = False)

## direction correction

In [34]:
def dir_name(x):
    if x==1:
        return 'E'
    elif x==-1:
        return 'W'
    elif x==2:
        return 'N'
    else:
        return 'S'
# E 1, W -1, N 2, S -2

In [35]:
def dir_id(x):
    if x[10]=='E':
        return 1
    elif x[10]=='W':
        return -1
    elif x[10]=='N':
        return 2
    else:
        return -2
# E 1, W -1, N 2, S -2

In [36]:
def direction_process(a):
    a['dir_id']=a['routeId'].apply(dir_id)
    a['opposite']=(-1)*a['dir_id']
    a['OPPOSITE_ROUTE']=a.routeId.str[0:10]+a['opposite'].apply(dir_name)
    
    a['direction_correction']=1    
    a.loc[(a['measure']<=a['END_MEASURE']) & (a['routeId']==a['END_ROUTEID']),'direction_correction']=0
    
    
    a = a.drop(['dir_id','opposite'],axis=1)
    
    a.loc[(a['direction_correction']==1) & ~pd.isnull(a.routeId.str[11]), 'direction_correction']=0 #exclude ramps

    return a

In [37]:
aa = direction_process(aa)

In [38]:
aa.tail(5)

,LABEL,XPOSITION,YPOSITION,HEADING,VELOCITY,ROADTEMP,AIRTEMP,SOLIDMATERIAL,LIQUIDMATERIAL,PREWETMATERIAL,...,measure,END_LABEL,END_LATI,END_LONG,END_DT,END_ROUTEID,END_MEASURE,route_correction,OPPOSITE_ROUTE,direction_correction
10924,A34393,-93.490700,41.150139,313,64.820,NaN,NaN,0,0,0,...,41.2213749886,A34393,41.151470,-93.492607,20161211161522,S001920065N,41.3567736428,0,S001920065S,0
10925,A34393,-93.492607,41.151470,313,64.820,NaN,NaN,0,0,0,...,41.3567736428,A34393,41.152691,-93.494324,20161211161533,S001920065N,41.4796119518,0,S001920065S,0
10926,A34393,-93.494324,41.152691,313,62.968,NaN,NaN,0,0,0,...,41.4796119518,A34393,41.153912,-93.496040,20161211161544,S001920065N,41.6025475543,0,S001920065S,0
10927,A34393,-93.496040,41.153912,313,64.820,NaN,NaN,0,0,0,...,41.6025475543,A34393,41.155048,-93.497627,20161211161554,S001920065N,41.7165728338,0,S001920065S,0
10929,B44042T,-91.609207,41.710369,60,9.260,NaN,NaN,0,0,0,...,0.137587994185,B44042T,41.710659,-91.609131,20161211160811,I072441010N,0.157640632914,0,I072441010S,0


In [39]:
aa.to_csv('direction_correction.csv', index = False)

In [46]:
def get_measure_w_direction_start(df):
    

    route='routeId'
    xx=df['XPOSITION']
    yy=df['YPOSITION']

            

    
    d = []
    loc = {'geometry':{'y':yy, 'x':xx}, 'routeId':df[route]}
    d.append(loc)

    locations = json.dumps(d, ensure_ascii=False)
       

    url = 'https://gis.iowadot.gov/ramsa/rest/services/lrs/MapServer/exts/LRSServer/networkLayers/0/geometryToMeasure?'
    tolerance = 50
    
    data = {'f':'json','locations': locations,'tolerance':str(tolerance),'inSR':'4326'}
    response = requests.post(url, data=data)
    r=response.json()
    
    if r['locations'][0]['status']!=u'esriLocatingOK':
        dat=None
    else:
        dat=(r['locations'][0]['results'][0]['measure'])

    
    return dat


In [47]:
def get_measure_w_direction_end(df):
    

    route='END_ROUTEID'
    xx=df['END_LONG']
    yy=df['END_LATI']

            

    
    d = []
    loc = {'geometry':{'y':yy, 'x':xx}, 'routeId':df[route]}
    d.append(loc)

    locations = json.dumps(d, ensure_ascii=False)
       

    url = 'https://gis.iowadot.gov/ramsa/rest/services/lrs/MapServer/exts/LRSServer/networkLayers/0/geometryToMeasure?'
    tolerance = 50
    
    data = {'f':'json','locations': locations,'tolerance':str(tolerance),'inSR':'4326'}
    response = requests.post(url, data=data)
    r=response.json()
    
    if r['locations'][0]['status']!=u'esriLocatingOK':
        dat=None
    else:
        dat=(r['locations'][0]['results'][0]['measure'])

    
    return dat


In [48]:
def direction_measure_correction(a):
    a.loc[(a['direction_correction']==1), 'routeId'] = a.loc[(a['direction_correction']==1), 'OPPOSITE_ROUTE']
    a.loc[(a['direction_correction']==1), 'END_ROUTEID'] = a.loc[(a['direction_correction']==1), 'OPPOSITE_ROUTE']
    
    a.loc[(a['direction_correction']==1), 'measure']=\
    a.loc[a['direction_correction']==1].apply(get_measure_w_direction_start, axis=1)
    a.loc[(a['direction_correction']==1), 'END_MEASURE']=\
    a.loc[a['direction_correction']==1].apply(get_measure_w_direction_end, axis=1)

    a = a.drop(['OPPOSITE_ROUTE'], axis=1)
    return a

In [49]:
aa = direction_measure_correction(aa)

In [51]:
aa.head(5)

,LABEL,XPOSITION,YPOSITION,HEADING,VELOCITY,ROADTEMP,AIRTEMP,SOLIDMATERIAL,LIQUIDMATERIAL,PREWETMATERIAL,...,routeId,measure,END_LABEL,END_LATI,END_LONG,END_DT,END_ROUTEID,END_MEASURE,route_correction,direction_correction
0,A29481,-92.408852,41.066711,0,0.0,NaN,NaN,0,0,0,...,M582542580N,0.988070900741,A29481,41.066719,-92.408859,20161211160013,M582542580N,0.988070900741,0,0
1,A29481,-92.408859,41.066719,0,0.0,NaN,NaN,0,0,0,...,M582542580N,0.988070900741,A29481,41.066711,-92.408852,20161211160016,M582542580N,0.988070900741,0,0
2,A29481,-92.408852,41.066711,0,0.0,NaN,NaN,0,0,0,...,M582542580N,0.988070900741,A29481,41.066711,-92.408852,20161211160026,M582542580N,0.988070900741,0,0
3,A29481,-92.408852,41.066711,0,0.0,NaN,NaN,0,0,0,...,M582542580N,0.988070900741,A29481,41.066711,-92.408852,20161211160037,M582542580N,0.988070900741,0,0
4,A29481,-92.408852,41.066711,0,0.0,NaN,NaN,0,0,0,...,M582542580N,0.988070900741,A29481,41.066711,-92.408852,20161211160048,M582542580N,0.988070900741,0,0


In [52]:
aa = aa.drop(['route_correction','direction_correction'], axis=1).reset_index(drop=True)

In [53]:
# save new file
aa.to_csv('15min_test_res_2.csv', index=False)